In [ ]:
import flopy as fp
import xarray
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl

In [ ]:
ds = xarray.load_dataset('../background/newMVTruth.nc')
ds

# Let's make the MCDonald Valley TRUTH model

In [ ]:
nrow = 40
ncol = 25
nlay = 5

top = np.ones((nrow, ncol)) * 100
botm = np.array([ds[f'bottom_layer{i}'].data for i in range(1,6)])
botm[0, :, :] = top - 50 
delr = np.full((ncol,), 500)
delc = np.full((nrow,), 500)
idomain = np.ones((nlay, nrow, ncol))
idomain[2,:,:] = ds.clay_location.data

modelgrid = fp.discretization.StructuredGrid(
    delc,
    delr,
    top,
    botm,
    idomain,
)
print(modelgrid)
print(type(modelgrid))

In [ ]:
sim_ws = pl.Path('../Truthmodel')
if sim_ws.exists():
    shutil.rmtree(sim_ws)

In [ ]:
sim = fp.mf6.MFSimulation(sim_ws=sim_ws)

In [ ]:
tdis = fp.mf6.ModflowTdis(sim, )

ims = fp.mf6.ModflowIms(sim, complexity='simple', outer_maximum=7000)

In [ ]:
gwf = fp.mf6.ModflowGwf(sim,modelname='mv')

In [ ]:
dis = fp.mf6.ModflowGwfdis(gwf, 
                           xorigin=0, 
                           yorigin=0, 
                           nlay=nlay, 
                           nrow=nrow,
                           ncol=ncol,
                           delr=delr, 
                           idomain=idomain, 
                           top=top, 
                           botm=botm
                          )

In [ ]:
ic = fp.mf6.ModflowGwfic(gwf, strt=11.0)

In [ ]:
k33 = np.ones((nlay, nrow, ncol))*.25
k33[2,:,:] = 1.0
npf = fp.mf6.ModflowGwfnpf(gwf, save_flows=True, k = np.array(
    [ds[f'k1_layer{i}'].data for i in [1,2]] +
    [ds['k1_layer3'].data] + 
    [ds[f'k1_layer{i}'].data for i in [4,5]]),
    k33overk=True,
    k33=k33
)

In [ ]:
oc = fp.mf6.ModflowGwfoc(gwf,
                        budget_filerecord='mv.cbc',
                        head_filerecord='mv.hds',
                        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
                        printrecord=[("HEAD", "ALL"), ("BUDGET", "ALL")])

In [ ]:
lake_array = ds.lake_location.data
lakeidx = np.where(lake_array==1)

chd = fp.mf6.ModflowGwfchd(gwf,
                           maxbound=len(lakeidx[0]),
       stress_period_data={0:[list(i) for i in list(zip(np.zeros(len(lakeidx[0])).astype(int),
                                 *lakeidx,
                                 np.ones(len(lakeidx[0]))*11))]}
                      )


In [ ]:
rch = fp.mf6.ModflowGwfrcha(gwf,
                            recharge=0.003641
                       )

In [ ]:
rivrows = list(range(22,40))


spd = [list(i) for i in list(zip(list(np.zeros(len(rivrows)).astype(int)),
        rivrows,
        list(np.ones(len(rivrows)).astype(int)*8),
        list(1.75-np.arange(len(rivrows))*.1),
        list(1.0e5*np.ones(len(rivrows))),
        list(2*np.ones(len(rivrows))*(-2)),
       ['pf']*2 + ['ds']*(len(rivrows)-2)
      ))]
spd

In [ ]:
riv = fp.mf6.ModflowGwfriv(gwf,
                           maxbound=len(spd),
                           boundnames=True,
                           stress_period_data={0:spd}
                          )

In [ ]:
wt_obs=[['wt01', 'head',  (0, 2, 17)], 
        ['wt02', 'head',  (0, 3, 10)],
        ['wt03', 'head',  (0, 6, 20)],
        ['wt04', 'head',  (0, 12, 22)],
        ['wt05', 'head',  (0, 14, 11)],
        ['wt06', 'head',  (0, 16, 18)],
        ['wt07', 'head',  (0, 17, 1)],
        ['wt08', 'head',  (0, 18, 6)], 
        ['wt09', 'head',  (0, 19, 11)], 
        ['wt10', 'head',  (0, 18, 22)], 
        ['wt11', 'head',  (0, 26, 5)],
        ['wt12', 'head',  (0, 27, 11)], 
        ['wt13', 'head',  (0, 28, 23)],
        ['wt14', 'head',  (0, 30, 6)], 
        ['wt15', 'head',  (0, 33, 14)],
        ['wt16', 'head',  (0, 36, 1)],
        ['wt17', 'head',  (0, 37, 22)]]

In [ ]:
aq_obs = [  
  ['aq01', 'head',  (4,	2, 17)],
  ['aq02', 'head',  (3,	3, 10)],
  ['aq03', 'head',  (3,	6, 20)],
  ['aq04', 'head',  (3,	12,	22)],
  ['aq05', 'head',  (4,	14,	11)],
  ['aq06', 'head',  (3,	16,	18)],
  ['aq07', 'head',  (3,	17,	1)],
  ['aq08', 'head',  (3,	18,	6)],
  ['aq09', 'head',  (3,	19,	11)],
  ['aq10', 'head',  (3,	18,	22)],
  ['aq11', 'head',  (3,	26,	5)],
  ['aq12', 'head',  (3,	27,	11)],
  ['aq13', 'head',  (3,	28,	23)],
  ['aq14', 'head',  (3,	30,	6)],
  ['aq15', 'head',  (4,	33,	14)],
  ['aq16', 'head',  (3,	36,	1)],
  ['aq17', 'head',  (3,	37,	22)]]

In [ ]:
obs = fp.mf6.ModflowUtlobs(gwf,continuous = {'wtobs.csv':wt_obs,
                                             'aquiferobs.csv':aq_obs})

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()